In [1]:

import torch
from binNF.normalizing_flows.manager import *
import matplotlib.pyplot as plt
import numpy as np

In [2]:
"""
def g(x):
    return torch.exp(-torch.sum(x**2,axis=-1)) #Vanilla Gaussian

def g(x): #ring, expect 0.51 in 2 dim
    q=torch.sqrt(torch.sum(x**2,axis=-1))
    #print(q)
    f=q.clone()
    a=torch.zeros_like(q)
    b=torch.ones_like(q)
    z=torch.where(q>0.45,a,q)
    #print(z)
    #print(torch.where(z>1,a,b))
    return torch.where(z<0.2,a,b)
"""

def g(x): #box: expect 16 in dim 2, 64 in dim 3
   
    q=torch.max(torch.abs(x),dim=-1).values
  
    a=torch.zeros_like(q)
    b=torch.ones_like(q)
   
    return torch.where(q>2,a,b)
"""
def tanp(x):
    return 100*(1+((torch.tan((x-0.5)*np.pi))**2))*np.pi   #derivative for jacobian

def gaussianp(x):
    mu=0
    sigma=1
    return torch.prod(-((x-mu)*torch.unsqueeze(torch.exp(-torch.sum((x-mu)**2/2*sigma**2,-1))/(np.sqrt(2*np.pi)*sigma**3),dim=-1)),-1)
"""
def normal(x,mu, sigma, n_flow):
    return (torch.exp(-torch.sum((x-mu)**2/(2*sigma**2),-1)))/(sigma*np.sqrt((2*np.pi)**n_flow))

n_flow=2
w = torch.empty((2000000,n_flow)) 
torch.nn.init.uniform_(w)
v=100*torch.tan((w-0.5)*(np.pi))
print(v)
print("Value of the integral before mapping by CL")
print(torch.mean(torch.mul(g(v),torch.abs(torch.prod(tanp(w),axis=-1)))))
#print(torch.var(torch.mul(g(v),torch.prod(tanp(w),axis=-1))))

NF =  AffineManager(n_flow=n_flow)
NF.create_model(n_pass_through=1,n_cells=1, nn_width=2, NN=[20,20,20], roll_step=1)
#print(v)
v=NF.model(NF.format_input(v))
jac=v[:,-1]
print(jac)
v=v[:,:n_flow]

mult=torch.mul(g(v),jac)

print("Value of the integral after mapping by CL")
print(torch.mean(torch.mul(mult,torch.prod(tanp(w),axis=-1))))


w = torch.empty((2000000,n_flow)) 
Z=torch.nn.init.normal_(w,std=1)

print("Value of the integral in gaussian")

print(torch.mean(g(Z)/(normal(Z,0,1,n_flow)))) 
#print(torch.var(torch.mul(g(v),torch.prod(tanp(w),axis=-1))))
# With Gaussian example:
#dim 2, cells 1: reproduces integral very good
#dim 2, cells 2: slightly more deviation, O(1e-3)
#dim 2, cells 10: O(1e-1)
#dim 4, cells 2: deviation O(1e-1)
#dim 2, cells 4: deviation O(1e-3)
#dim 2, cells 8: deviation O(1e-3)
#dim 8, cells 2: deviation O(10)
#dim 8, cells 4: deviation O(10)
#result stable up to dim 5 -> sampling to sparse in high dimension, no bug


tensor([[1086.3318,  115.5336],
        [  46.9078,  -66.4197],
        [  99.0206,  -59.1204],
        ...,
        [-178.1409,   76.2800],
        [   6.9047, -156.6460],
        [ -60.3211,  -99.9049]])
Value of the integral before mapping by CL
tensor(37.2064)


TypeError: forward() missing 1 required positional argument: 'dev'

In [5]:
def g(x): #box: expect 0.25 in dim2
   
    q=torch.max(torch.abs(x),dim=-1).values
  
    a=torch.zeros_like(q)
    b=torch.ones_like(q)
   
    return torch.where(q>0.5,a,b)


n_flow=2
w = torch.empty((150000,n_flow)) 
torch.nn.init.uniform_(w)

print("Value of the integral before mapping by CL")
print(torch.mean(g(w)))

NF =  PWQuadManager(n_flow=n_flow)
NF.create_model(n_cells=2, n_bins=10, NN=[20,20,20,20,20], dev=torch.device("cpu"))
#print(v)
v=NF.model(NF.format_input(w,torch.device("cpu")))
jac=v[:,-1]
print(jac)
v=v[:,:n_flow]

mult=torch.mul(g(v),jac)
print("Value of the integral after mapping by CL")
print(torch.mean(mult))

Value of the integral before mapping by CL
tensor(0.2492)
tensor([1.2398, 0.7747, 1.3923,  ..., 0.8232, 0.9215, 1.2262],
       grad_fn=<SelectBackward>)
Value of the integral after mapping by CL
tensor(0.2492, grad_fn=<MeanBackward0>)
